# Sorting Pandas dataframes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enabling-languages/python-i18n/blob/main/notebooks/sorting_pandas2.ipynb)

This notebook looks at a number of alternatives for sorting natural language content in a Pandas dataframe.

For further information on sorting, please read Enabling Languages's [introduction to collation](https://github.com/enabling-languages/python-i18n/blob/main/notebooks/Collation.ipynb)

__TODO__: add support for language specific casing to normalisation functions in language tailored sorts.

## Setup

__For Colab__: the setup code will need to install the required locales. Once the locales are installed, it is necessary to restart the runtime environment and run all code blocks.


In [28]:
import locale
import pandas as pd, unicodedata as ud
# import unicodedataplus as ud

try:
  import google.colab
  IN_COLAB = True
except ImportError:
  IN_COLAB = False
if IN_COLAB:
  !pip install PyICU
  try:
    locale.setlocale(locale.LC_ALL, "fr_FR.UTF-8")
  except locale.Error:
    print("Currently installing required locale. After the install the runtime environment will restart, please rerun all code.")
    !sudo apt-get install language-pack-fr
    import os
    os.kill(os.getpid(), 9)
else:
  locale.setlocale(locale.LC_ALL, "")

## Load data

In [29]:
data = {
    'lexème': ["zèbre", "Être", "Zéro", "Déblayer", "éveillé", "dessaisir", "déranger", "écouler", "effacer", "à gauche", "Asiatique", "exact"],
    'nombre': [11, 23, 16, 30, 14, 35, 13, 22, 26, 9, 31, 27]
}

df = pd.DataFrame(data)

# are_equivalent(list1, list2) or are_equivalent(string1, string2)
#    compares two lists or two strings for cannonical equivalence.
#
def are_equivalent(i1, i2):
    if isinstance(i1, list) and isinstance(i2, list):
      return [ud.normalize("NFD", e) for e in i1] == [ud.normalize("NFD", e) for e in i2]
    elif isinstance(i1, str) and isinstance(i2, str):
      return ud.normalize("NFD", i1) == ud.normalize("NFD", i2)
    return

Sorting Options:

1. ISO/IEC 14651 (default) python sort: `df.sort_values(column)`
2. Normalised ISO/IEC 14651 sort: `df.sort_values(column, key = lambda x: x.str.normalize("NFKC").str.lower())` or `df.sort_values(column, key=lambda x: normalised_sort(x, nf="NFKC"))` \
  Can specify normalisation form: NFC, FKC, NFD, NFKD.
3. POSIX based locale sort: `df.sort_values(column, key=lambda x: x.map(locale.strxfrm))`
4. Normalised POSIX based locale sort:  `df.sort_values(column, key=lambda x: x.str.normalize("NFKC").str.lower().map(locale.strxfrm))` or `df.sort_values(column, key=lambda x: normalised_sort(x, nf="NFKC", loc=True))` \
  Can specify normalisation form: NFC, FKC, NFD, NFKD.
5. PyICU

Ensure that there are two dataframes one using Unicode Normalisation Form C (NFC), and the other using Unicode Normalisation Form D (NFD).

In [30]:
df_nfc = df.copy()
df_nfc["lexème"] = df_nfc["lexème"].str.normalize('NFC')
df_nfd = df.copy()
df_nfd["lexème"] = df_nfd["lexème"].str.normalize('NFD')

## 1) ISO/IEC 14651 - default python sort

This is the standard `df.sort_values()` operation, without any modifications, an untailored ISO/IEC 14651 sort.

In [31]:
df_nfc_default_sort = df_nfc.sort_values("lexème")
nfc_default = list(df_nfc_default_sort["lexème"])

df_nfd_default_sort = df_nfd.sort_values("lexème")
nfd_default = list(df_nfd_default_sort["lexème"])

print(nfc_default)
print(nfd_default)

print(are_equivalent(nfc_default, nfd_default))

['Asiatique', 'Déblayer', 'Zéro', 'dessaisir', 'déranger', 'effacer', 'exact', 'zèbre', 'Être', 'à gauche', 'écouler', 'éveillé']
['Asiatique', 'Déblayer', 'Être', 'Zéro', 'à gauche', 'dessaisir', 'déranger', 'effacer', 'exact', 'écouler', 'éveillé', 'zèbre']
False


`df.sort_values` performs a codepoint ordering based on ISO/IEC 14651. Pandas sorting aligns with the underlying Python sorting algorithms. A number of features can be observed:

1. All basic Latin uppercase characters are sorted before their lowercase equivalents.
2. Characters are approximately ordered according to their codepoint order
3. Unicode normalisation will directly affect the order of letters with diacritics, i.e. __á__ would sort after __z__ when it is a precomposed sequence, but would sort after __az__ when it is a decomposed sequence.
4. the sort algorithms do not support language tailoring directly.

We can adjust the sort behaviour through the `key` parameter.

First we will define helper functions:

In [32]:
# normalised_sort()
#    Performs unicode normalisation and lowercasing prior to sorting.
def normalised_sort(s, nf="NFKD", loc=False):
    nf = nf.upper()
    if nf in ["NFC", "NFKC", "NFD", "NFKD"]:
        if isinstance(s, pd.core.series.Series):
            s = s.str.normalize(nf).str.lower().map(locale.strxfrm) if loc else s.str.normalize(nf).str.lower()
        else:
            s = locale.strxfrm(ud.normalize(nf, s).lower()) if loc else ud.normalize(nf, s).lower()
    return s


## 2) Normalised ISO/IEC 14651 sort

In [33]:
df_nfc_custom_sort = df_nfc.sort_values("lexème", key = lambda x: x.str.normalize("NFKC").str.lower())
nfc_custom = list(df_nfc_custom_sort["lexème"])

df_nfd_custom_sort = df_nfd.sort_values("lexème", key = lambda x: x.str.normalize("NFKC").str.lower())
nfd_custom = list(df_nfd_custom_sort["lexème"])

print(nfc_custom)
print(nfd_custom)

print(are_equivalent(nfc_custom, nfd_custom))

['Asiatique', 'dessaisir', 'Déblayer', 'déranger', 'effacer', 'exact', 'zèbre', 'Zéro', 'à gauche', 'écouler', 'éveillé', 'Être']
['Asiatique', 'dessaisir', 'Déblayer', 'déranger', 'effacer', 'exact', 'zèbre', 'Zéro', 'à gauche', 'écouler', 'éveillé', 'Être']
True


In [34]:
df_nfc_custom_sort2 = df_nfc.sort_values("lexème", key = lambda x: normalised_sort(x, nf="NFKC"))
nfc_custom2 = list(df_nfc_custom_sort2["lexème"])

df_nfd_custom_sort2 = df_nfd.sort_values("lexème", key = lambda x: normalised_sort(x, nf="NFKC"))
nfd_custom2 = list(df_nfd_custom_sort2["lexème"])

print(nfc_custom2)
print(nfd_custom2)

print(are_equivalent(nfc_custom2, nfd_custom2))

['Asiatique', 'dessaisir', 'Déblayer', 'déranger', 'effacer', 'exact', 'zèbre', 'Zéro', 'à gauche', 'écouler', 'éveillé', 'Être']
['Asiatique', 'dessaisir', 'Déblayer', 'déranger', 'effacer', 'exact', 'zèbre', 'Zéro', 'à gauche', 'écouler', 'éveillé', 'Être']
True


The `key` function `normalised_sort()` provides more consistent results, and closer to user expectations.:

## 3) POSIX based locale sort


In [35]:
df_nfc_locale_sort = df_nfc.sort_values("lexème", key=lambda x: x.map(locale.strxfrm))
nfc_locale = list(df_nfc_locale_sort["lexème"])

df_nfd_locale_sort = df_nfd.sort_values("lexème", key=lambda x: x.map(locale.strxfrm))
nfd_locale = list(df_nfd_locale_sort["lexème"])

print(nfc_locale)
print(nfd_locale)

print(are_equivalent(nfc_locale, nfd_locale))

['Asiatique', 'Déblayer', 'Être', 'Zéro', 'à gauche', 'déranger', 'dessaisir', 'écouler', 'effacer', 'éveillé', 'exact', 'zèbre']
['Asiatique', 'Déblayer', 'Être', 'Zéro', 'à gauche', 'dessaisir', 'déranger', 'effacer', 'exact', 'écouler', 'éveillé', 'zèbre']
False


A POSIX based locale sort will vary depending on the operating system being used. 

Different workstations and servers will have access to different sets of locales, depending on operating system, the OS version and in some cases what the sys admins or devops have made available.

Additionally, for many locales on macOS, the LC_COLLATE table, for many locales, is symlinked to a [single collation table](https://gist.github.com/andjc/50db7bd920ad1953e14944046e0f681a). This will mean that locale based sorts on macOS will differ from the same locale based sort on other platforms.

For instance on macOS the above code yields:

```py
['Asiatique', 'Déblayer', 'Être', 'Zéro', 'à gauche', 'déranger', 'dessaisir', 'écouler', 'effacer', 'éveillé', 'exact', 'zèbre']
['Asiatique', 'Déblayer', 'Être', 'Zéro', 'à gauche', 'dessaisir', 'déranger', 'effacer', 'exact', 'écouler', 'éveillé', 'zèbre']
False
```

While on Google Colab (Ubuntu 18.04):

```py
['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
True
```

## 4) Normalised POSIX based locale sort

Uses the `df_sort()` function to apply normalised language tailored sorting, using the locale module.

In [36]:
df_nfc_locale_custom_sort = df_nfc.sort_values("lexème", key=lambda x: x.str.normalize("NFKC").str.lower().map(locale.strxfrm))
nfc_locale_custom = list(df_nfc_locale_custom_sort["lexème"])

df_nfd_locale_custom_sort = df_nfd.sort_values("lexème", key=lambda x: x.str.normalize("NFKC").str.lower().map(locale.strxfrm))
nfd_locale_custom = list(df_nfd_locale_custom_sort["lexème"])

print(nfc_locale_custom)
print(nfd_locale_custom)
print(are_equivalent(nfc_locale_custom, nfd_locale_custom))

['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
True


Alternatively:

In [37]:
df_nfc_locale_custom_sort2 = df_nfc.sort_values("lexème", key=lambda x: normalised_sort(x, nf="NFKC", loc=True))
nfc_locale_custom2 = list(df_nfc_locale_custom_sort2["lexème"])

df_nfd_locale_custom_sort2 = df_nfd.sort_values("lexème", key=lambda x: normalised_sort(x, nf="NFKC", loc=True))
nfd_locale_custom2 = list(df_nfd_locale_custom_sort2["lexème"])

print(nfc_locale_custom2)
print(nfd_locale_custom2)
print(are_equivalent(nfc_locale_custom2, nfd_locale_custom2))

['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
True


## 5) PyICU based sorting

Provides language tailored sorting using PyICU's wrappers around `icu4c`.

In [38]:
from icu import Locale, Collator
loc = Locale("fr_FR")
collator = Collator.createInstance(loc)

df_nfc_icu_sort = df_nfc.sort_values("lexème", key=lambda x: x.map(collator.getSortKey))
nfc_icu = list(df_nfc_icu_sort["lexème"])

df_nfd_icu_sort = df_nfd.sort_values("lexème", key=lambda x: x.map(collator.getSortKey))
nfd_icu = list(df_nfd_icu_sort["lexème"])

print(nfc_icu)
print(nfd_icu)
print(are_equivalent(nfc_icu, nfd_icu))

['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
['à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro']
True


__ICU__ normalises internally as part of sorting process, so there is no need to normalise for ICU sorting.

## Reflections

__ICU__ and the normalised sorting techniques provide consistent results for canonically equivalent text.

|Technique  |Result |
|---------- |------ |
|Normalised default |'Asiatique', 'à gauche', 'dessaisir', 'Déblayer', 'déranger', 'effacer', 'exact', 'écouler', 'éveillé', 'Être', 'zèbre', 'Zéro' |
|Normalised POSIX locale |'à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro' |
|ICU |'à gauche', 'Asiatique', 'Déblayer', 'déranger', 'dessaisir', 'écouler', 'effacer', 'Être', 'éveillé', 'exact', 'zèbre', 'Zéro' |